## 文档读取

各类文档加载示例代码

### 目录
- txt文档加载器
- md文档加载器
- pdf文档加载器
- word文档加载器

---

### 1. txt读取

In [ ]:
# txt loader
# author: Dusizhong
# since: 2025-03-03

"""
python=3.10
pip install langchian
langchain-community
"""

doc_path = './files/test.txt'
# doc_path = './files/test.md'

# load txt
def load_txt(doc_path):
    with open(doc_path, 'r') as file:
        text = file.read()
        return text
        
print('text: \n\n', load_txt(doc_path))

# 2. load md
# pip install
from langchain_community.document_loaders.text import TextLoader
def load_md(doc_path):
    loader = TextLoader(doc_path)
    doc = loader.load()
    return doc
# print('doc: \n\n', load_md(doc_path))



### 2. pdf读取

---

In [ ]:
# pdf loader
# author: Dusizhong
# since: 2025-03-03

# pip install 

from langchain_community.document_loaders import PyPDFLoader

doc_path = r'./files/正式招标文件.pdf'

async def load_pdf(doc_path):
    loader = PyPDFLoader(doc_path)
    pages = []
    async for page in loader.alazy_load():
        pages.append(page)
    return pages
    
doc = await load_pdf(doc_path)
print(f"{doc[0].metadata}\n")
print(doc[0].page_content)

### 3. word读取

In [24]:
# 提取word全部文本
# 按行遍历，包含文本和表格，表格字段以制表符分割
# @author: sizhong du
# @since: 2025-03-03


doc_path = './files/招标文件-唐山乐亭绿色交通车储一体化储能电站项目设备采购（定稿）.docx'

def load_docx(doc_path):
    doc = WordDocument(doc_path)
    tables = doc.tables
    table_idx = 0
    full_text = []

    for element in doc.element.body:
        if element.tag.endswith('p'):
            para_text = element.text.strip()
            full_text.append(para_text)
        elif element.tag.endswith('tbl'):
            if table_idx < len(tables):
                for row in tables[table_idx].rows:
                    row_content = [cell.text.strip() if cell.text else '' for cell in row.cells]
                    row_text = '\t'.join(row_content)
                    full_text.append(row_text)
                    # print(row_text)
                    table_idx += 1
    
    return '\n'.join(full_text)

full_text = load_docx(doc_path)
print('full_text', len(full_text))

full_text 51480


In [4]:
# 按章节提取word内容
# 按行遍历内容，包含文本和表格，表格字段以制表符分割
# @author: sizhong du
# @since: 2025-03-13

import re
import os
from docx import Document

class ChapterExtractor:

    def __init__(self, doc_path):
        self.doc = Document(doc_path)
        self.chapters = []

    def is_chapter_title(self, text):
        patterns = [
            r'^第[一二三四五六七八九十\d]+章',
            # r'^\d+\.\d+',
            # r'^[IVXLCDM]+\.'
        ]
        return any(re.match(pattern, text) for pattern in patterns) if patterns else False

    def extract(self):
        tables = self.doc.tables
        table_idx = 0
        current_chapter = None

        for element in self.doc.element.body:
            if element.tag.endswith('p'):
                text = element.text.strip()
                if self.is_chapter_title(text):
                    if current_chapter is not None:
                        self.chapters.append(current_chapter)
                    current_chapter = {'title': text, 'paragraphs': [], 'tables': []}
                else:
                    if current_chapter is None:
                        current_chapter = {'title': '前言', 'paragraphs': [], 'tables': []}
                    current_chapter['paragraphs'].append(text)
            elif element.tag.endswith('tbl'):
                if current_chapter is None:
                    current_chapter = {'title': '前言', 'paragraphs': [], 'tables': []}
                if table_idx < len(tables):
                    table_data = []
                    for row in tables[table_idx].rows:
                        row_text = [cell.text.strip() if cell.text else '' for cell in row.cells]
                        table_data.append('\t'.join(row_text))
                    current_chapter['tables'].extend(table_data)
                    table_idx += 1
        
        # add last chapter
        if current_chapter is not None:
            self.chapters.append(current_chapter)

        return self.chapters

    # save file
    def save_file(self, output_dir="output"):
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        for idx, (title, content) in enumerate(self.sections):
            if not title:
                filename = f"section_{idx+1}.txt"
            else:
                filename = re.sub(r'[\\/:*?"<>|]', '_', title)[:100] + ".txt"
            
            filepath = os.path.join(output_dir, filename)
            with open(filepath, 'w', encoding='utf-8') as f:
                f.write("\n".join(content))
            print(f"已保存章节：{filepath}")


if __name__ == "__main__":
    # doc_path = './files/test.docx'
    # doc_path = r'./files/1. 河北省材料采购招标文件示范文本(试行).docx'
    # doc_path = './files/（定稿）河北税务2025年税务专网线路租用采购招标文件.doc'
    doc_path = './files/招标文件-唐山乐亭绿色交通车储一体化储能电站项目设备采购（定稿）.docx'
    extractor = ChapterExtractor(doc_path)
    chapters = extractor.extract()
    for chapter in chapters:
        print(chapter)


33
{'title': '前言', 'paragraphs': ['', '', '', '唐山乐亭绿色交通车储一体化储能电站项目设备采购', '', '', '', '招  标  文  件', '', '招标编号：HBCT-250461-001', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '招   标   人：河钢工业技术服务有限公司', '招标代理机构：河北省成套招标有限公司', '', '编制日期：二0二五年三月', '', '', '说明：', '本部分招标文件除封面外共计 95 页，投标单位在领取招标文件时请自行核对，如发现缺页、乱码等情况，自收到招标文件之日起三日内向招标人或招标代理机构提出，联系电话： 0311-83086927 。如未提出，所有责任及由此造成的一切后果由投标单位自负。'], 'tables': []}
52
{'title': '第一章  招标公告', 'paragraphs': ['', '唐山乐亭绿色交通车储一体化储能电站项目设备采购', '招标公告', '1.招标条件', '本招标项目 唐山乐亭绿色交通车储一体化储能电站项目设备采购 已由 / 以 / 批准建设，项目业主为 河钢工业技术服务有限公司 ，建设资金来自 企业自筹 ，出资比例为 100 ，招标人为 河钢工业技术服务有限公司 。项目已具备招标条件，现对该项目进行公开招标。', '2.项目概况与招标范围', '2.1项目概况：利用厂区原有地块建设车储一体化储能电站，本项目移动储能系统装机容量40MW/79MWh，其中车储一体化电池箱35MW/70MWh，移动补能车容量5MW/9MWh，电气一、二次设备，运营管理平台。', '2.1.1项目名称：唐山乐亭绿色交通车储一体化储能电站项目设备采购', '2.1.2招标编号：HBCT-250461-001', '2.1.3设备交货及安装地点：河北乐亭县招标人指定地点。', '2.2招标范围：唐山乐亭绿色交通车储一体化储能电站项目设备采购。具体内容详见第五章“技术要求”。', '2.3交货期：合同签订后45日内设备供应、设备安装调试完成，具备实现项目投运。', '2.4采购设备要求：设备质量及保障年限应符合甲方要求。', '3.投标人资格要求', '3.

### 以下为调试中代码，无法正常使用

---

In [5]:
# 按章节提取word内容2
# 包含多级子章节，注意：未包含表格
# @author: sizhong du
# @since: 2025-03-03

# pip install python-docx

from docx import Document as WordDocument

doc_path = r'./files/1. 河北省材料采购招标文件示范文本(试行).docx'
# doc_path = './files/test.docx'

# 提取全部文本（含表格）
def load_docx(doc_path):
    doc = WordDocument(doc_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    for table in doc.tables:
        for row in table.rows:
            row_text = []
            for cell in row.cells:
                row_text.append(cell.text)
            full_text.append('\t'.join(row_text)) # 使用制表符作为表格单元格的分隔符
    full_content = '\n'.join(full_text)
    return full_content

# full_content = load_docx(doc_path)
# print('full_content', full_content)

# 按章节提取（含子章节）
def load_chapters(doc_path):
    doc = WordDocument(doc_path)
    found_chapter = False
    chapters = []
    chapter = {'title': '', 'content': []}
    for paragraph in doc.paragraphs:
        if paragraph.style.name.startswith('Heading 1'):
            level = int(paragraph.style.name.split(' ')[-1])
            if(chapter['title']):
                chapters.append(chapter)
            chapter = {'title': paragraph.text.strip(), 'content': []}
        else:
            chapter['content'].append(paragraph.text)
    if(chapter['title']):
        chapters.append(chapter)
    return chapters

# chapters = load_chapters(doc_path)
# for chapter in chapters:
#     print(chapter)

# 按标题提取
chapter_title = '1.招标条件'
def extract_chapter_paragraphs(doc_path, chapter_title):
    doc = WordDocument(doc_path)
    found_chapter = False
    paragraphs = []

    for paragraph in doc.paragraphs:
        if paragraph.style.name.startswith('Heading') and paragraph.text.strip() == chapter_title:
            found_chapter = True
            continue
        
        if found_chapter:
            if paragraph.style.name.startswith('Heading'):
                break
            else:
                paragraphs.append(paragraph.text)

    return paragraphs

# 多级提取
def extract_chapters(doc_path):
    document = WordDocument(doc_path)
    chapters = []
    current_section = None
    
    for paragraph in document.paragraphs:
        if paragraph.style.name.startswith('Heading'):
            level = int(paragraph.style.name.split(' ')[-1])
            section = {'level': level, 'title': paragraph.text.strip(), 'content': []}
            
            # Find the correct parent to append this section
            if not current_section or level <= current_section['level']:
                while current_section and level <= current_section['level']:
                    current_section = current_section.get('parent')
                if current_section:
                    current_section.setdefault('children', []).append(section)
                else:
                    chapters.append(section)
                section['parent'] = current_section
            else:
                current_section.setdefault('children', []).append(section)
                section['parent'] = current_section
            
            current_section = section
        elif current_section:
            current_section['content'].append(paragraph.text.strip())
    return chapters

# paragraphs = extract_chapter_paragraphs(doc_path, chapter_title)
paragraphs = extract_chapters(doc_path)
for i, paragraph in enumerate(paragraphs, start=1):
    print(f"Paragraph {i}: {paragraph}")


KeyboardInterrupt: 

In [27]:
# python实现读取word文档，按章节保存内容。
# 要求：提供多种判断是否为章节方式，包括不限于样式和章节关键字等，
# 以及其他各种情况，最大程度确保提取准确性。

# 实现思路
# 1.章节的判断条件：章节标题可能使用特定的样式，比如“标题1”、“标题2”等。另外，可能有些文档直接用“第一章”、“第一节”或者“1.1”这样的关键字开头。
# 2.遍历文档中的段落，检查每个段落是否是标题样式。比如，如果段落的style.name是“Heading 1”或类似，那么这就是一个章节标题。
#   同时，还要检查段落的文本是否以预定义的关键字开头，比如“第x章”、“第x节”等。
# 3.考虑是否需要保存为嵌套的结构，因为章节有层级结构。比如，一级标题、二级标题等。
# 4.处理特殊情况：有些章节标题可能没有使用样式，但符合关键字；或者有些段落虽然有关键字但不是章节，比如在正文中出现“第一章”这个词。

import re
import os
from docx import Document

class ChapterExtractor:
    def __init__(self, doc_path):
        self.doc = Document(doc_path)
        self.sections = []
        self.current_title = None
        self.current_content = []

    def is_heading_style(self, paragraph, styles):
        return paragraph.style.name in styles if styles else False

    def starts_with_keyword(self, text, keywords):
        return any(text.startswith(keyword) for keyword in keywords) if keywords else False

    def matches_regex(self, text, patterns):
        return any(re.match(pattern, text) for pattern in patterns) if patterns else False

    # 判断是否为章节标题
    # 接受段落作为参数，返回是否是章节标题及标题级别
    def is_chapter_title(self, paragraph, config):
        text = paragraph.text.strip()
        return any([
            self.is_heading_style(paragraph, config.get('styles')),
            self.starts_with_keyword(text, config.get('keywords')),
            self.matches_regex(text, config.get('patterns'))
        ])

    def extract(self, config=None):

        # 定义章节标题判定规则
        if config is None:
            config = {
                'styles': ['Heading 1'],
                'keywords': ['第', '章', '节'],
                'patterns': [
                    r'^第[一二三四五六七八九十\d]+章', # 匹配“第一章”、“第1章”等
                    r'^\d+\.\d+', # 匹配“1.1”、“2.3.4”等
                    r'^[IVXLCDM]+\.' # 匹配I. V. IX.
                ]
            }

        # 处理文档开头的无标题内容
        self.sections = []
        pre_content = []

        for para in self.doc.paragraphs:
            if self.is_chapter_title(para, config):
                if self.current_title is None:
                    pre_content = self.current_content.copy()
                else:
                    self.sections.append((self.current_title, self.current_content))
                self.current_title = para.text.strip()
                self.current_content = []
            else:
                if para.text.strip():
                    self.current_content.append(para.text)

        # 处理最后部分
        if self.current_title:
            self.sections.append((self.current_title, self.current_content))
        elif pre_content:
            self.sections.append(("前言", pre_content + self.current_content))

        return self.sections


    def save_sections(self, output_dir="output"):
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        for idx, (title, content) in enumerate(self.sections):
            # 生成安全文件名
            if not title:
                filename = f"section_{idx+1}.txt"
            else:
                filename = re.sub(r'[\\/:*?"<>|]', '_', title)[:100] + ".txt"
            
            filepath = os.path.join(output_dir, filename)
            with open(filepath, 'w', encoding='utf-8') as f:
                f.write("\n".join(content))
            print(f"已保存章节：{filepath}")


if __name__ == "__main__":
    # 使用示例
    doc_path = './files/招标文件-唐山乐亭绿色交通车储一体化储能电站项目设备采购（定稿）.docx'
    extractor = ChapterExtractor(doc_path)
    chapters = extractor.extract()
    for chapter in chapters:
        print(chapter)
    
    # 自定义配置（根据需要修改）
    config = {
        'styles': ['Heading 1', 'Title'],
        'keywords': ['第', '章', '节'],
    }



('第一章  招标公告', ['唐山乐亭绿色交通车储一体化储能电站项目设备采购', '招标公告', '1.招标条件', '本招标项目 唐山乐亭绿色交通车储一体化储能电站项目设备采购 已由 / 以 / 批准建设，项目业主为 河钢工业技术服务有限公司 ，建设资金来自 企业自筹 ，出资比例为 100 ，招标人为 河钢工业技术服务有限公司 。项目已具备招标条件，现对该项目进行公开招标。', '2.项目概况与招标范围'])
('2.1项目概况：利用厂区原有地块建设车储一体化储能电站，本项目移动储能系统装机容量40MW/79MWh，其中车储一体化电池箱35MW/70MWh，移动补能车容量5MW/9MWh，电气一、二次设备，运营管理平台。', [])
('2.1.1项目名称：唐山乐亭绿色交通车储一体化储能电站项目设备采购', [])
('2.1.2招标编号：HBCT-250461-001', [])
('2.1.3设备交货及安装地点：河北乐亭县招标人指定地点。', [])
('2.2招标范围：唐山乐亭绿色交通车储一体化储能电站项目设备采购。具体内容详见第五章“技术要求”。', [])
('2.3交货期：合同签订后45日内设备供应、设备安装调试完成，具备实现项目投运。', [])
('2.4采购设备要求：设备质量及保障年限应符合甲方要求。', ['3.投标人资格要求'])
('3.1具有独立法人资格，有良好信誉和经济实力，并具有与本招标内容相应的供货能力。', [])
('3.2财务要求：投标人须提供经会计师事务所或审计机构审计的2023年度财务审计报告，无资不抵债情况（投标人的成立时间少于规定年份的，应提供成立以来的财务状况表或基本账户银行出具的资信证明文件）。', [])
('3.3业绩要求：近年（2023年3月1日至投标截止时间，以合同签订时间为准）至少有有一项合同金额不少于1000万元的储能设备供货业绩。', [])
('3.4信誉要求：未被“信用中国网站”（www.creditchina.gov.cn）列入失信被执行人、重大税收违法失信主体、政府采购严重违法失信名单。', [])
('3.5本次招标不接受联合体投标。', [])
('3.6单位负责人为同一人或者存在控股、管理关系的不同单位，不得参加同一标段投标或者未划分标段的同一招标项目投标。', ['4.投标报名及招标

In [4]:
# langchain unstructed file loader
# author Dusizhong
# since: 2025-03-04

# 安装：
# 1.简单安装：
# pip install langchain-community unstructured[docx]
# 2.完整安装：
# pip install "unstructured[local-inference]"
# 选装：Layout Detection模块主要是对文档图片进行目标检测识别，比如使用Faster R-CNN、Mask R-CNN。
# pip install "detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2"
# 选装：LayoutParser是基于Detectron2提供最小的接口,是一个版面分析工具包，它提供了布局检测、OCR识别、布局分析等接口
# pip install layoutparser[layoutmodels,tesseract]
# 3.安装依赖：
# nltk库：NLTK库高效自然语言工具包
# pip install nltk
# 自动下载数据包：(报错：getaddrinfo failed，参考：https://blog.csdn.net/m0_59222901/article/details/143824224)
# import nltk
# nltk.download()
# 手动下载数据包：
# https://github.com/nltk/nltk_data，将packages改名nltk_data放置根目录下（查看数据路径print(nltk.data.path) 


# 示例：
# doc_path = r'./files/1. 河北省材料采购招标文件示范文本(试行).docx'
# doc_path = './files/（定稿）河北税务2025年税务专网线路租用采购招标文件.doc' #暂无法加载
doc_path = './files/招标文件-唐山乐亭绿色交通车储一体化储能电站项目设备采购（定稿）.docx'
# doc_path = './files/test.docx'

# 使用UnstructuredWordDocumentLoader加载
# from langchain_community.document_loaders import UnstructuredWordDocumentLoader
# loader = UnstructuredWordDocumentLoader(doc_path)
# documents = loader.load()
# print(documents)
# print(len(documents[0].page_content))

# 使用UnstructuredFileLoader加载
from langchain.document_loaders import UnstructuredFileLoader
loader = UnstructuredFileLoader(
    doc_path,
    mode="elements",  # 按文档元素分块
    include_header=False,  # 关闭页眉解析
    include_footer=False,  # 关闭页脚解析
    chunking_strategy="by_title",  # 按标题分块
    max_characters=4000,  # 单块最大字符数
)
chunks = loader.load()
print(chunks)

[Document(metadata={'source': './files/招标文件-唐山乐亭绿色交通车储一体化储能电站项目设备采购（定稿）.docx', 'emphasized_text_contents': ['说明：', '唐山乐亭绿色交通车储一体化储能电站项目设备采购', '招标公告', '1.', '招标条件', '2.', '项目概况与招标范围', '3.', '投标人资格要求', '4.', '投标报名及招标文件的获取', '5.', '投标文件的递交', '6', '.发布公告的媒介', '7', '. 其他公示内容', '8. 提出异议渠道和方式', '9. 本招标项目的监督部门', '10. 招标人或者其委托的招标代理机构使用的第三方交易平台的付费主体及收费标准', '11.联系方式'], 'emphasized_text_tags': ['b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b'], 'file_directory': './files', 'filename': '招标文件-唐山乐亭绿色交通车储一体化储能电站项目设备采购（定稿）.docx', 'last_modified': '2025-03-12T10:34:38', 'orig_elements': 'eJztXVtvJMtt/iuCnmIgpa17FffJcBAgL87TCfxgGYu67go5qz3e1fr4xEh+e0hW90yPNJJmZvvAG6dfWqO+VLOKLJIfyar+49+u24/tY7t/eHdXr99eXXvTY+vdC1dtEjbrLGKqTQTtKmjZmlby+p+vrj+2h1TTQ8Jn/nZd0kN7/+nzL+9q++nhA56SeEe/+7G9q3efW3nAS9T2zRs69+V6unifPjY6ffvV65rxGGXAo4t4tA1JuP3qnJJ4LEHRKSfpmOrt14Ak3n6NOmg+Y2+/glQJz3Tv8X6ZIp2XdH93Bs8YtTsfZacWnHF4TBXw2RjwGHxreDVnPDogUkAVPMaq8f7eJT7rMiR6ynQ+Azf1U/kr